+ __说明:__

本程序是 阿里云天池 的比赛 津南数字制造算法挑战赛(赛场一) B 榜20名提交结果所使用的最终程序

队伍名称 Drop, 作者: 陶亚凡 陕西科技大学

我的 github 和 博客:(虽然没东西把, 但是还是求波关注, follow, stars /捂脸)

github: https://github.com/taoyafan

博客: https://me.csdn.net/taoyafan

队友（Blue 电子科技大学的） github 和 博客：(也求波关注, follow, stars /捂脸)

github：https://github.com/BluesChang

博客：https://blueschang.github.io

程序各个部分很大程度的参考了鱼佬的 baseline

感谢队友的很多贡献, 感谢鱼佬和他的的 baseline, 感谢林有夕大佬让我在群里不停的学到新知识.

因为我这是第一了 ML 的比赛, 看着鱼佬 baseline 开始学习 pandas, sklearn 还有相关知识, 所以水平实在有限. 希望各位大佬给点意见或建议, 有什么问题或者可以改进的地方请告知我, 灰常感谢. 

一直想开源, 但是成绩太差, 趁着还在首页, 赶快开源了, 我也是抱着学习的心态, 也没想着拿奖, 所以这个程序也没啥保留~

+ __程序思路:__ 

读取数 -> 手动处理训练集明显异常数据 -> 数据清洗 -> 特征工程 -> 训练

__数据清洗:__

删除缺失率过高的数据 -> 处理字符时间(段) -> 计算时间差 -> 处理异常值 -> 删除单一类别占比过大的特征

__特征工程:__

构建新特征 -> 利用特征之间的相关性预测 nan 值 -> 后向特征选择

__训练:__

使用 lgb 和 xgb 自动选择最优参数, 然后融合

+ __数据通路:__

开始学鱼佬的 baseline 自己写着写着变量名太多了,前后运行总是出现各种小错误, 所以对整体结构改了好多次, 最终使用了 pipe line, 包括了整个数据清洗和特征工程部分, 所以变量名少了, 各个部分也不存在耦合了, 所以我觉得有必要介绍下数据通路:

数据读取得到 train, test ----> 合并得到 full ---> 经过 pipe line 得到 pipe_data ---> 训练集测试集分割得到 X_train 和 X_test ---> 训练得到结果 oof 和 predictions



# 导入包，读取数据

In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go 
import plotly.tools as tls
from xgboost import XGBRegressor
from tpot import TPOTRegressor

/usr/local/lib/python3.6/dist-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning:

Falling back to the python version of hypervolume module. Expect this to be very slow.

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning:

Not importing directory /usr/local/lib/python3.6/dist-packages/mpl_toolkits: missing __init__

/usr/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning:

Not importing directory /usr/local/lib/python3.6/dist-packages/google: missing __init__



In [2]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RepeatedKFold, ShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import log_loss
from sklearn.preprocessing import Imputer
from scipy.stats import skew
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__



In [3]:
py.init_notebook_mode(connected=True)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

## 设定文件名, 读取文件

In [4]:
train_file_name = 'data/jinnan_round1_train_20181227.csv'
test_file_name = 'data/jinnan_round1_testB_20190121.csv'
test_name = 'testB'

In [5]:
# 读取数据， 改名
train = pd.read_csv(train_file_name, encoding = 'gb18030')
test  = pd.read_csv(test_file_name, encoding = 'gb18030')
train.rename(columns={'样本id':'id', '收率':'target'}, inplace = True)
test.rename(columns={'样本id':'id', '收率':'target'}, inplace = True)
target_name = 'target'

# 存在异常数据，改为 nan
train.loc[1304, 'A25'] = np.nan
train['A25'] = train['A25'].astype(float)

# 去掉 id 前缀
train['id'] = train['id'].apply(lambda x: int(x.split('_')[1]))
test['id'] = test['id'].apply(lambda x: int(x.split('_')[1]))

train.drop(train[train[target_name] < 0.87].index, inplace=True)
full=pd.concat([train, test], ignore_index=True)

# 数据清洗

## 删除缺失率高的特征

+ __删除缺失值大于 th_high 的值__
+ __缺失值在 th_low 和 th_high 之间的特征根据是否缺失增加新特征__
  
  如 B10 缺失较高，增加新特征 B10_null，如果缺失为1，否则为0

In [6]:
class del_nan_feature(BaseEstimator, TransformerMixin):
    
    def __init__(self, th_high=0.85, th_low=0.02):
        self.th_high = th_high
        self.th_low = th_low
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'del_nan_feature', ' '*5, '-'*30, '\n')
        print("shape before process = {}".format(X.shape))

        # 删除高缺失率特征
        X.dropna(axis=1, thresh=(1-self.th_high)*X.shape[0], inplace=True)
        
        
        # 缺失率较高，增加新特征
        for col in X.columns:
            if col == 'target':
                continue
            
            miss_rate = X[col].isnull().sum()/ X.shape[0]
            if miss_rate > self.th_low:
                print("Missing rate of {} is {:.3f} exceed {}, adding new feature {}".
                     format(col, miss_rate, self.th_low, col+'_null'))
                X[col+'_null'] = 0
                X.loc[X[pd.isnull(X[col])].index, [col+'_null']] = 1
        print("shape = {}".format(X.shape))

        return X

## 处理字符时间（段）

In [7]:
# 处理时间
def timeTranSecond(t):
    try:
        h,m,s=t.split(":")
    except:

        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif pd.isnull(t):
            return np.nan
        else:
            return 0

    try:
        tm = (int(h)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600

    return tm

In [8]:
# 处理时间差
def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+",se)
#         print("sh, sm, eh, em = {}, {}, {}, {}".format(sh, em, eh, em))
    except:
        if pd.isnull(se):
            return np.nan, np.nan, np.nan

    try:
        t_start = (int(sh)*3600 + int(sm)*60)/3600
        t_end = (int(eh)*3600 + int(em)*60)/3600
        
        if t_start > t_end:
            tm = t_end - t_start + 24
        else:
            tm = t_end - t_start
    except:
        if se=='19:-20:05':
            return 19, 20, 1
        elif se=='15:00-1600':
            return 15, 16, 1
        else:
            print("se = {}".format(se))


    return t_start, t_end, tm

In [9]:
class handle_time_str(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'handle_time_str', ' '*5, '-'*30, '\n')

        for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
            try:
                X[f] = X[f].apply(timeTranSecond)
            except:
                print(f,'应该在前面被删除了！')


        for f in ['A20','A28','B4','B9','B10','B11']:
            try:
                start_end_diff = X[f].apply(getDuration)
                
                X[f+'_start'] = start_end_diff.apply(lambda x: x[0])
                X[f+'_end'] = start_end_diff.apply(lambda x: x[1])
                X[f] = start_end_diff.apply(lambda x: x[2])

            except:
                print(f,'应该在前面被删除了！')
        return X

## 计算时间差

In [10]:
def t_start_t_end(t):
    if pd.isnull(t[0]) or pd.isnull(t[1]):
#         print("t_start = {}, t_end = {}, id = {}".format(t[0], t[1], t[2]))
        return np.nan
        
    if t[1] < t[0]:
        t[1] += 24
        
    dt = t[1] - t[0]

    if(dt > 24 or dt < 0):
#         print("dt error, t_start = {}, t_end = {}, id = {}".format(t[0], t[1], t[2]))
        return np.nan
    
    return dt

In [11]:
class calc_time_diff(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'calc_time_diff', ' '*5, '-'*30, '\n')

        # t_start 为时间的开始， tn 为中间的时间，减去 t_start 得到时间差
        t_start = ['A9', 'A24', 'B5']
        tn = {'A9':['A11', 'A14', 'A16'], 'A24':['A26'], 'B5':['B7']}
        
        # 计算时间差
        for t_s in t_start:
            for t_e in tn[t_s]:
                X[t_e+'-'+t_s] = X[[t_s,t_e, target_name]].apply(t_start_t_end, axis=1)
                
        # 所有结果保留 3 位小数
        X = X.apply(lambda x:round(x, 3))
        
        print("shape = {}".format(X.shape))
        
        return X

## 处理异常值

+ __单一类别个数小于 threshold 的值视为异常值, 改为 nan__

In [12]:
class handle_outliers(BaseEstimator, TransformerMixin):

    def __init__(self, threshold=2):
        self.th = threshold
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'handle_outliers', ' '*5, '-'*30, '\n')
        category_col = [col for col in X if col not in ['id', 'target']]
        for col in category_col:
            label = X[col].value_counts(dropna=False).index.tolist()
            for i, num in enumerate(X[col].value_counts(dropna=False).values):
                if num <= self.th:
#                     print("Number of label {} in feature {} is {}".format(label[i], col, num))
                    X.loc[X[col]==label[i], [col]] = np.nan
        
        print("shape = {}".format(X.shape))
        return X

## 删除单一类别占比过大特征

In [13]:
class del_single_feature(BaseEstimator, TransformerMixin):

    def __init__(self, threshold=0.98):
        # 删除单一类别占比大于 threshold 的特征
        self.th = threshold
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'del_single_feature', ' '*5, '-'*30, '\n')
        category_col = [col for col in X if col not in ['target']]
        
        for col in category_col:
            rate = X[col].value_counts(normalize=True, dropna=False).values[0]
            
            if rate > self.th:
                print("{} 的最大类别占比是 {}, drop it".format(col, rate))
                X.drop(col, axis=1, inplace=True)

        print("shape = {}".format(X.shape))
        return X

# 特征工程

## 获得训练集与测试集

In [14]:
def split_data(pipe_data, target_name='target'):
   
    # 特征列名
    category_col = [col for col in pipe_data if col not in ['target',target_name]]
    
    # 训练、测试行索引
    train_idx = pipe_data[np.logical_not(pd.isnull(pipe_data[target_name]))].index
    test_idx = pipe_data[pd.isnull(pipe_data[target_name])].index
    
    # 获得 train、test 数据
    X_train = pipe_data.loc[train_idx, category_col].values.astype(np.float64)
    y_train = np.squeeze(pipe_data.loc[train_idx, [target_name]].values.astype(np.float64))
    X_test = pipe_data.loc[test_idx, category_col].values.astype(np.float64)
    
    return X_train, y_train, X_test, test_idx

## xgb(用于特征 nan 值预测)

In [15]:
##### xgb
def xgb_predict(X_train, y_train, X_test, params=None, verbose_eval=100):
    
    if params == None:
        xgb_params = {'eta': 0.05, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 
                  'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
    else:
        xgb_params = params

    folds = KFold(n_splits=10, shuffle=True, random_state=2018)
    oof_xgb = np.zeros(len(X_train))
    predictions_xgb = np.zeros(len(X_test))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        if(verbose_eval):
            print("fold n°{}".format(fold_+1))
            print("len trn_idx  {}".format(len(trn_idx)))
            
        trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
        val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

        watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
        clf = xgb.train(dtrain=trn_data,
                        num_boost_round=20000,
                        evals=watchlist,
                        early_stopping_rounds=200,
                        verbose_eval=verbose_eval,
                        params=xgb_params)
        
        
        oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
        predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits

    if(verbose_eval):
        print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train)))
    return oof_xgb, predictions_xgb

## 根据 B14 构建新特征

In [16]:
class add_new_features(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print('-'*30, ' '*5, 'add_new_features', ' '*5, '-'*30, '\n')

        # 经过测试,只有 B14 / B12 有用
        
#         X['B14/A1'] = X['B14'] / X['A1']
#         X['B14/A3'] = X['B14'] / X['A3']
#         X['B14/A4'] = X['B14'] / X['A4']
#         X['B14/A19'] = X['B14'] / X['A19']
#         X['B14/B1'] = X['B14'] / X['B1']
#         X['B14/B9'] = X['B14'] / X['B9']

        X['B14/B12'] = X['B14'] / X['B12']
        
        print("shape = {}".format(X.shape))
        return X

## 选择特征, nan 值填充

+ __选择可能有效的特征__   (只是为了加快选择时间)

+ __利用其他特征预测 nan，取最近值填充__

In [17]:
def get_closest(indexes, predicts):
    print("From {}".format(predicts))

    for i, one in enumerate(predicts):
        predicts[i] = indexes[np.argsort(abs(indexes - one))[0]]

    print("To {}".format(predicts))
    return predicts
    

def value_select_eval(pipe_data, selected_features):
    
    # 经过多次测试, 只选择可能是有用的特征
    cols_with_nan = [col for col in pipe_data.columns 
                     if pipe_data[col].isnull().sum()>0 and col in selected_features]

    for col in cols_with_nan:
        X_train, y_train, X_test, test_idx = split_data(pipe_data, target_name=col)
        oof_xgb, predictions_xgb = xgb_predict(X_train, y_train, X_test, verbose_eval=False)
        
        print("-"*100, end="\n\n")
        print("CV normal MAE scores of predicting {} is {}".
              format(col, mean_absolute_error(oof_xgb, y_train)/np.mean(y_train)))
        
        pipe_data.loc[test_idx, [col]] = get_closest(pipe_data[col].value_counts().index,
                                              predictions_xgb)

    pipe_data = pipe_data[selected_features+['target']]

    return pipe_data

# pipe_data = value_eval(pipe_data)

In [18]:
class selsected_fill_nans(BaseEstimator, TransformerMixin):

    def __init__(self, selected_features = ['A3_null', 'A6', 'A16', 'A25', 'A28', 'A28_end',
                                           'B5', 'B10_null', 'B11_null', 'B14', 'B14/B12', 'id']):
        self.selected_fearutes = selected_features
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'selsected_fill_nans', ' '*5, '-'*30, '\n')

        X = value_select_eval(X, self.selected_fearutes)

        print("shape = {}".format(X.shape))
        return X

In [19]:
def modeling_cross_validation(data):
    X_train, y_train, X_test, test_idx = split_data(data,
                                                    target_name='target')
    oof_xgb, _ = xgb_predict(X_train, y_train, X_test, verbose_eval=False)
    print('-'*100, end='\n\n')
    return mean_squared_error(oof_xgb, y_train)


def featureSelect(data):

    init_cols = [f for f in data.columns if f not in ['target']]
    best_cols = init_cols.copy()
    best_score = modeling_cross_validation(data[best_cols+['target']])
    print("初始 CV score: {:<8.8f}".format(best_score))

    for col in init_cols:
        best_cols.remove(col)
        score = modeling_cross_validation(data[best_cols+['target']])
        print("当前选择特征: {}, CV score: {:<8.8f}, 最佳cv score: {:<8.8f}".
              format(col, score, best_score), end=" ")
        
        if best_score - score > 0.0000004:
            best_score = score
            print("有效果,删除！！！！")
        else:
            best_cols.append(col)
            print("保留")

    print('-'*100)
    print("优化后 CV score: {:<8.8f}".format(best_score))
    return best_cols, best_score

## 后向选择特征

In [20]:
class select_feature(BaseEstimator, TransformerMixin):

    def __init__(self, init_features = None):
        self.init_features = init_features
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print('-'*30, ' '*5, 'select_feature', ' '*5, '-'*30, '\n')
        
        if self.init_features:
            X = X[self.init_features + ['target']]
            best_features = self.init_features
        else:
            best_features = [col for col in X.columns]
        
        last_feartues = []
        iteration = 0
        equal_time = 0
        
        best_CV = 1
        best_CV_feature = []
        
        # 打乱顺序,但是使用相同种子,保证每次运行结果相同
        np.random.seed(2018)
        while True:
            print("Iteration = {}\n".format(iteration))
            best_features, score = featureSelect(X[best_features + ['target']])
            
            # 保存最优 CV 的参数
            if score < best_CV:
                best_CV = score
                best_CV_feature = best_features
                print("Found best score :{}, with features :{}".format(best_CV, best_features))
                
            np.random.shuffle(best_features)
            print("\nCurrent fearure length = {}".format(len(best_features)))
            
            # 最终 3 次迭代相同，则终止迭代
            if len(best_features) == len(last_feartues):
                equal_time += 1
                if equal_time == 3:
                    break
            else:
                equal_time = 0
            
            last_feartues = best_features
            iteration = iteration + 1

            print("\n\n\n")
            
        return X[best_features + ['target']]


# 训练

## 构建 pipeline, 处理数据

In [21]:
selected_features = ['A3_null', 'A6', 'A16', 'A25', 'A28', 'A28_end', 
                     'B5', 'B10_null', 'B11_null', 'B14', 'B14/B12', 'id']

pipe = Pipeline([
                ('del_nan_feature', del_nan_feature()),
                ('handle_time_str', handle_time_str()),
                ('calc_time_diff', calc_time_diff()),
                ('Handle_outliers', handle_outliers(2)),
                ('del_single_feature', del_single_feature(1)),
                ('add_new_features', add_new_features()),
                ('selsected_fill_nans', selsected_fill_nans(selected_features)),
                ('select_feature', select_feature(selected_features)),
                ])

pipe_data = pipe.fit_transform(full.copy())
print(pipe_data.shape)

------------------------------       del_nan_feature       ------------------------------ 

shape before process = (1532, 44)
Missing rate of A3 is 0.029 exceed 0.02, adding new feature A3_null
Missing rate of B10 is 0.172 exceed 0.02, adding new feature B10_null
Missing rate of B11 is 0.597 exceed 0.02, adding new feature B11_null
shape = (1532, 44)
------------------------------       handle_time_str       ------------------------------ 

A7 应该在前面被删除了！
------------------------------       calc_time_diff       ------------------------------ 

shape = (1532, 61)
------------------------------       handle_outliers       ------------------------------ 

shape = (1532, 61)
------------------------------       del_single_feature       ------------------------------ 

shape = (1532, 61)
------------------------------       add_new_features       ------------------------------ 

shape = (1532, 62)
------------------------------       selsected_fill_nans       ------------------------------ 

----------------------------------------------------------------------------------------------------

当前选择特征: B14/B12, CV score: 0.00012291, 最佳cv score: 0.00011808 保留
----------------------------------------------------------------------------------------------------

当前选择特征: A28_end, CV score: 0.00011717, 最佳cv score: 0.00011808 有效果,删除！！！！
----------------------------------------------------------------------------------------------------

当前选择特征: B5, CV score: 0.00011785, 最佳cv score: 0.00011717 保留
----------------------------------------------------------------------------------------------------

当前选择特征: A6, CV score: 0.00012231, 最佳cv score: 0.00011717 保留
----------------------------------------------------------------------------------------------------

当前选择特征: A16, CV score: 0.00011826, 最佳cv score: 0.00011717 保留
----------------------------------------------------------------------------------------------------

当前选择特征: B10_null, CV score: 0.00011966, 最佳cv score: 0.00011717 保留
---

----------------------------------------------------------------------------------------------------

当前选择特征: B14, CV score: 0.00012226, 最佳cv score: 0.00011747 保留
----------------------------------------------------------------------------------------------------
优化后 CV score: 0.00011747

Current fearure length = 8




Iteration = 6

----------------------------------------------------------------------------------------------------

初始 CV score: 0.00011715
----------------------------------------------------------------------------------------------------

当前选择特征: id, CV score: 0.00018398, 最佳cv score: 0.00011715 保留
----------------------------------------------------------------------------------------------------

当前选择特征: A3_null, CV score: 0.00011984, 最佳cv score: 0.00011715 保留
----------------------------------------------------------------------------------------------------

当前选择特征: A25, CV score: 0.00012132, 最佳cv score: 0.00011715 保留
----------------------------------------------

## 自动调参

In [22]:
def find_best_params(pipe_data, predict_fun, param_grid):
    
    # 获得 train 和 test, 归一化
    X_train, y_train, X_test, test_idx = split_data(pipe_data, target_name='target')
    min_max_scaler = MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X_test = min_max_scaler.transform(X_test)
    best_score = 1

    # 遍历所有参数,寻找最优
    for params in ParameterGrid(param_grid):
        print('-'*100, "\nparams = \n{}\n".format(params))

        oof, predictions = predict_fun(X_train, y_train, X_test, params=params, verbose_eval=False)
        score = mean_squared_error(oof, y_train)
        print("CV score: {}, current best score: {}".format(score, best_score))

        if best_score > score:
            print("Found new best score: {}".format(score))
            best_score = score
            best_params = params


    print('\n\nbest params: {}'.format(best_params))
    print('best score: {}'.format(best_score))
    
    return best_params

## lgb

In [23]:
def lgb_predict(X_train, y_train, X_test, params=None, verbose_eval=100):
    
    if params == None:
        lgb_param = {'num_leaves': 20, 'min_data_in_leaf': 2, 'objective':'regression', 'max_depth': 4,
         'learning_rate': 0.06, "min_child_samples": 3, "boosting": "gbdt", "feature_fraction": 1,
         "bagging_freq": 0.7, "bagging_fraction": 1, "bagging_seed": 11, "metric": 'mse', "lambda_l2": 0.003,
         "verbosity": -1}
    else :
        lgb_param = params
        
    folds = KFold(n_splits=10, shuffle=True, random_state=2018)
    oof_lgb = np.zeros(len(X_train))
    predictions_lgb = np.zeros(len(X_test))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        if verbose_eval:
            print("fold n°{}".format(fold_+1))
            
        trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
        val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

        num_round = 10000
        clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data],
                        verbose_eval=verbose_eval, early_stopping_rounds = 100)
        
        oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)

        predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

        if verbose_eval:
            print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, y_train)))
    
    return oof_lgb, predictions_lgb

+ __选择最优参数__

In [24]:
param_grid = [
        {'num_leaves': [20], 'min_data_in_leaf': [2, 3], 'objective':['regression'],
         'max_depth': [3, 4, 5], 'learning_rate': [0.06, 0.12, 0.24], "min_child_samples": [3],
         "boosting": ["gbdt"], "feature_fraction": [0.7], "bagging_freq": [1],
         "bagging_fraction": [1], "bagging_seed": [11], "metric": ['mse'],
         "lambda_l2": [0.0003, 0.001, 0.003], "verbosity": [-1]}
              ]

lgb_best_params = find_best_params(pipe_data, lgb_predict, param_grid)

---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.0003, 'learning_rate': 0.06, 'max_depth': 3, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 2, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00011489364207207567, current best score: 1
Found new best score: 0.00011489364207207567
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.0003, 'learning_rate': 0.06, 'max_depth': 3, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 3, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00012002848815037457, current best score: 0.00011489364207207567
--------

CV score: 0.00011435307236140136, current best score: 0.00011231611613708764
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.0003, 'learning_rate': 0.24, 'max_depth': 5, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 3, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00012009711500307733, current best score: 0.00011231611613708764
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.001, 'learning_rate': 0.06, 'max_depth': 3, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 2, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00011480005750940053,

CV score: 0.00012091648099518892, current best score: 0.00011212722352757958
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.001, 'learning_rate': 0.24, 'max_depth': 5, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 2, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00011431927180539175, current best score: 0.00011212722352757958
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.001, 'learning_rate': 0.24, 'max_depth': 5, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 3, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00012002843916496843, 

CV score: 0.000114353078699699, current best score: 0.00011212722352757958
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.003, 'learning_rate': 0.24, 'max_depth': 4, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 3, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.0001212472410367223, current best score: 0.00011212722352757958
---------------------------------------------------------------------------------------------------- 
params = 
{'bagging_fraction': 1, 'bagging_freq': 1, 'bagging_seed': 11, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l2': 0.003, 'learning_rate': 0.24, 'max_depth': 5, 'metric': 'mse', 'min_child_samples': 3, 'min_data_in_leaf': 2, 'num_leaves': 20, 'objective': 'regression', 'verbosity': -1}

CV score: 0.00011433021385951927, cur

+ __lgb 训练__

In [25]:
X_train, y_train, X_test, test_idx = split_data(pipe_data, target_name='target')
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)
oof_lgb, predictions_lgb = lgb_predict(X_train, y_train, X_test, params=lgb_best_params, verbose_eval=200) #

fold n°1
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[84]	training's l2: 7.64635e-05	valid_1's l2: 0.00013957
CV score: 0.76865147
fold n°2
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[82]	training's l2: 8.03721e-05	valid_1's l2: 0.000110882
CV score: 0.68291002
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 5.30193e-05	valid_1's l2: 0.000139552
Early stopping, best iteration is:
[173]	training's l2: 5.68838e-05	valid_1's l2: 0.000138392
CV score: 0.59707720
fold n°4
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 5.73108e-05	valid_1's l2: 0.000104267
Early stopping, best iteration is:
[104]	training's l2: 7.56046e-05	valid_1's l2: 9.52206e-05
CV score: 0.51161256
fold n°5
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 5.36518e-05	valid_1's l2: 0.000110701
Early 

## xgb

+ __选择最优参数__

In [26]:
param_grid = [
              {'silent': [1],
               'nthread': [4],
               'eval_metric': ['rmse'],
               'eta': [0.03],
               'objective': ['reg:linear'],
               'max_depth': [4, 5, 6],
               'num_round': [1000],
               'subsample': [0.4, 0.6, 0.8, 1],
               'colsample_bytree': [0.7, 0.9, 1],
               }
              ]

xgb_best_params = find_best_params(pipe_data, xgb_predict, param_grid)

---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 0.7, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 4, 'nthread': 4, 'num_round': 1000, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.4}

CV score: 0.00011466648461334117, current best score: 1
Found new best score: 0.00011466648461334117
---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 0.7, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 4, 'nthread': 4, 'num_round': 1000, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.6}

CV score: 0.00011456634369353389, current best score: 0.00011466648461334117
Found new best score: 0.00011456634369353389
---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 0.7, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 4, 'nthread': 4, 'num_roun

CV score: 0.00011510995861245286, current best score: 0.00011350220852811235
---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 0.9, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 6, 'nthread': 4, 'num_round': 1000, 'objective': 'reg:linear', 'silent': 1, 'subsample': 1}

CV score: 0.00011452153482808672, current best score: 0.00011350220852811235
---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 1, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 4, 'nthread': 4, 'num_round': 1000, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.4}

CV score: 0.00011658284174959551, current best score: 0.00011350220852811235
---------------------------------------------------------------------------------------------------- 
params = 
{'colsample_bytree': 1, 'eta': 0.03, 'eval_metric': 'rmse', 'max_depth': 4, 'nthread': 4, 'num_ro

+ __xgb 训练__

In [27]:
X_train, y_train, X_test, test_idx = split_data(pipe_data, target_name='target')
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)
oof_xgb, predictions_xgb = xgb_predict(X_train, y_train, X_test, params=xgb_best_params, verbose_eval=200) #

fold n°1
len trn_idx  1244
[0]	train-rmse:0.41223	valid_data-rmse:0.414495
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[200]	train-rmse:0.00937	valid_data-rmse:0.012154
[400]	train-rmse:0.00738	valid_data-rmse:0.012149
Stopping. Best iteration:
[249]	train-rmse:0.008639	valid_data-rmse:0.011963

fold n°2
len trn_idx  1244
[0]	train-rmse:0.412554	valid_data-rmse:0.411489
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[200]	train-rmse:0.009572	valid_data-rmse:0.010408
[400]	train-rmse:0.007492	valid_data-rmse:0.010257
Stopping. Best iteration:
[321]	train-rmse:0.008054	valid_data-rmse:0.010154

fold n°3
len trn_idx  1244
[0]	train-rmse:0.412511	valid_data-rmse:0.412077
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train 

## 模型融合

In [28]:
# stacking
train_stack = np.vstack([oof_lgb, oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack, y_train)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train[val_idx]
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
final_score = mean_squared_error(y_train, oof_stack)
print(final_score)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
0.00011094962448042872


# 生成提交结果

+ __生成文件名__

In [29]:
import time
model_name = "lgb_xgb"
file_name = 'result/{}_{}_{:5.0f}_{}features_{}.csv'.format(
                                        test_name, 
                                        model_name,
                                        final_score*1e8, X_train.shape[1],
                                        time.strftime('%Y%m%d_%H:%M:%S',time.localtime(time.time())))
                                        
print(file_name)

result/testB_lgb_xgb_11095_8features_20190122_10:36:07.csv


+ __写入文件__

In [30]:
sub_df = pd.read_csv(test_file_name, encoding = 'gb18030')
sub_df = sub_df[['样本id', 'A1']]
sub_df['A1'] = predictions
sub_df['A1'] = sub_df['A1'].apply(lambda x:round(x, 3))
sub_df.to_csv(file_name, header=0, index=0) 